In [1]:
import importlib
import functions

importlib.reload(functions) 

from functions import *

In [2]:
BASE_LINK = 'https://data.who.int/countries'

In [3]:
driver = start_browser()

In [4]:
open_page(driver, BASE_LINK)

In [5]:
counteries = get_list_of_countries(driver)

In [10]:
print(f"Total {len(counteries)} countries found")

Total 194 countries found


## NOW use only 1st country present in counteries list and write the required functions

In [6]:
country = counteries[0]
open_page(driver,country)

## NAJAM

**Things to do**
- 1 Name of country
- 2 Current health expenditure (% of GDP)
- 3 WHO region
- 4 World Bank income level
- 5 Population growth rate
- 6 Leading cause of death
- 7 Leading cause of under 5 mortality
- 8 extract data from population graph from 2000 to 2024 (get twenty rows w.r.t two columns year and population)
- 9 extract data from life expentency at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))
- 10 extract data from Healthy life expectancy (HALE) at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))


In [ ]:
# get_country_name(driver)

# get_current_health_expenditure(driver)

# get_who_region(driver)

# get_world_bank_income_level(driver)

# get_population_growth_rate(driver)

# get_leading_cause_of_death(driver)

# get_leading_cause_of_death(driver)

# get_leading_cause_of_under_5_mortality(driver)

# get_population_data(driver)

# get_life_expentency_data(driver)

# get_health_life_expectancy_data(driver)

## SHAHZAD

**Things to do**


- 3 two functions to extract **visit indicator** links for following categories (divide them in group A and B)
   - 3.1 Number of new HIV infections
   - 3.2 People living with tuberculosis (TB)
   - 3.3 Malaria cases
   - 3.4 Probability of dying from non-communicable diseases
   - 3.5 Road traffic deaths
   - 3.6 Suicide deaths
   - 3.7 Prevalence of hypertension
   - 3.8 Adult obesity
   - 3.9 Tobacco use
   - 3.10 Alcohol consumption
   - 3.11 Safely managed sanitation
   - 3.12 Fine particulate matter
   - 3.13 UHC index score
   - 3.14 Births attended by skilled health personnel
   - 3.15 Family planning
   - 3.16 DTP3 immunization
   - 3.17 MCV2 immunization
   - 3.18 Interventions against NTDs
   - 3.19 Density of doctors
   - 3.20 Density of nurses
   - 3.21 Density of pharmacists
   - 3.22 Density of dentists
   - 3.23 WASH development assistance
   - 3.24 Financial hardship
- 4 A general function to extract visit indicators of this kind of pages (https://data.who.int/indicators/i/49AC786/77D059C?m49=004)
   - 4.1 return 24 rows with two columns year and count.
- 5 a general function to extract the other kind of function
   - 5.1 get 23-24 years data from line graph displayed on indicator page (example https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004)
   

In [7]:
import importlib
import functions

importlib.reload(functions) 

from functions import *

group_A_indidcator=get_group_A_indicators(driver)

Button:  Health status

Header Matched:  Number of new HIV infections
Added Link:  https://data.who.int/indicators/i/49AC786/77D059C?m49=004
Skipped Header: People living with tuberculosis (TB)
Skipped Header: Malaria cases

Header Matched:  Probability of dying from non-communicable diseases
Added Link:  https://data.who.int/indicators/i/C540135/1F96863?m49=004
Skipped Header: Road traffic deaths

Header Matched:  Suicide deaths
Added Link:  https://data.who.int/indicators/i/F08B4FD/16BBF41?m49=004
-----------------
Button:  Risk factors

Header Matched:  Prevalence of hypertension
Added Link:  https://data.who.int/indicators/i/7DA4E68/608DE39?m49=004

Header Matched:  Adult obesity
Added Link:  https://data.who.int/indicators/i/C6262EC/BEFA58B?m49=004

Header Matched:  Tobacco use
Added Link:  https://data.who.int/indicators/i/847662C/75DDA77?m49=004

Header Matched:  Alcohol consumption
Added Link:  https://data.who.int/indicators/i/EF38E6A/EE6F72A?m49=004

Header Matched:  Safely m

In [8]:
import importlib
import functions

importlib.reload(functions) 

from functions import *
group_B_indidcator=get_group_B_indicators(driver)

Button:  Health status
Skipped Header: Number of new HIV infections

Header Matched:  People living with tuberculosis (TB)
Added Link:  https://data.who.int/indicators/i/13B4226/C288D13?m49=004

Header Matched:  Malaria cases
Added Link:  https://data.who.int/indicators/i/B868307/442CEA8?m49=004
Skipped Header: Probability of dying from non-communicable diseases

Header Matched:  Road traffic deaths
Added Link:  https://data.who.int/indicators/i/B9D9E6A/D6176E2?m49=004
Skipped Header: Suicide deaths
-----------------
Button:  Risk factors
Skipped Header: Prevalence of hypertension
Skipped Header: Adult obesity
Skipped Header: Tobacco use
Skipped Header: Alcohol consumption
Skipped Header: Safely managed sanitation
Skipped Header: Fine particulate matter
-----------------
Button:  Service coverage

Header Matched:  UHC index score
Added Link:  https://data.who.int/indicators/i/3805B1E/9A706FD?m49=004

Header Matched:  Births attended by skilled health personnel
Added Link:  https://data

In [9]:
open_page(driver, BASE_LINK)

In [13]:
import importlib
import functions

importlib.reload(functions) 


from functions import *
data_A=get_indicator_data_for_group_A(driver,group_A_indidcator)

Starting to fetch
Selected: 1990
0
Selected: 1991
0
Selected: 1992
0
Selected: 1993
0
Selected: 1994
0
Selected: 1995
0
Selected: 1996
0
Selected: 1997
0
Selected: 1998
0
Selected: 1999
0
Selected: 2000
0
Selected: 2001
0
Selected: 2002
0
Selected: 2003
0
Selected: 2004
0
Selected: 2005
0
Selected: 2006
0
Selected: 2007
0
Selected: 2008
0
Selected: 2009
0
Selected: 2010
0
Selected: 2011
0
Selected: 2012
0
Selected: 2013
0
Selected: 2014
0
Selected: 2015
0
Selected: 2016
0
Selected: 2017
0
Selected: 2018
0
Selected: 2019
0
Selected: 2020
0
Selected: 2021
0
Selected: 2022
0
Selected: 2000
35%
Selected: 2001
35%
Selected: 2002
35%
Selected: 2003
35%
Selected: 2004
35%
Selected: 2005
41%
Selected: 2006
41%
Selected: 2007
40%
Selected: 2008
39%
Selected: 2009
38%
Selected: 2010
38%
Selected: 2011
37%
Selected: 2012
37%
Selected: 2013
36%
Selected: 2014
36%
Selected: 2015
36%
Selected: 2016
36%
Selected: 2017
35%
Selected: 2018
35%
Selected: 2019
35%
Selected: 2000
4.1
Selected: 2001
4.9
Sel

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/section/div[2]/div[3]/div[1]/div[1]/div/div/div/div/div/div/div[1]/div[2]/div/div[2]/label/span[2]"}
  (Session info: MicrosoftEdge=129.0.2792.65); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C50ECF25+12997]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6C536BCE4+1897876]
	(No symbol) [0x00007FF6C4E9F1CC]
	(No symbol) [0x00007FF6C4EE672E]
	(No symbol) [0x00007FF6C4EE67B5]
	(No symbol) [0x00007FF6C4F24ED7]
	(No symbol) [0x00007FF6C4F07F8F]
	(No symbol) [0x00007FF6C4EDC09D]
	(No symbol) [0x00007FF6C4F22887]
	(No symbol) [0x00007FF6C4F07BB3]
	(No symbol) [0x00007FF6C4EDB5AC]
	(No symbol) [0x00007FF6C4EDAA8D]
	(No symbol) [0x00007FF6C4EDB171]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6C52

In [23]:
for i in data_A:
    print(i)

In [316]:
import importlib
import functions

importlib.reload(functions) 


from functions import *
data_B=get_indicator_data_for_group_B(driver,group_B_indidcator)

Starting to fetch


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/section/div[2]/div[3]/div[1]/div[1]/div/div/div/div/div/div/div[2]/div[2]/div[2]/div[4]/div/div/table/tbody"}
  (Session info: MicrosoftEdge=129.0.2792.65); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C50ECF25+12997]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6C536BCE4+1897876]
	(No symbol) [0x00007FF6C4E9F1CC]
	(No symbol) [0x00007FF6C4EE672E]
	(No symbol) [0x00007FF6C4EE67B5]
	(No symbol) [0x00007FF6C4F24ED7]
	(No symbol) [0x00007FF6C4F07F8F]
	(No symbol) [0x00007FF6C4EDC09D]
	(No symbol) [0x00007FF6C4F22887]
	(No symbol) [0x00007FF6C4F07BB3]
	(No symbol) [0x00007FF6C4EDB5AC]
	(No symbol) [0x00007FF6C4EDAA8D]
	(No symbol) [0x00007FF6C4EDB171]
	Microsoft::Applications::Events::EventProperty::empty [0x00

In [25]:
# get_group_A_indicators(driver)
def get_group_A_indicators(driver):
    url = "https://data.who.int/indicators" 
    open_page(driver,url)
    
    group_A_links = []

    for i in range(1,25):
        path=f"/html/body/main/section/div[2]/div/div[1]/div[1]/div/div[1]"
        indicator=driver.find_element(By.XPATH,path).text.split("\n")
        



        

    indicators = driver.find_elements(By.CSS_SELECTOR, '/html/body/main/section/div[2]/div/div[1]/div[1]/div/div[1]') 
    
    for indicator in indicators:
        link = indicator.get_attribute('href')
        group_A_links.append(link)
    
    return group_A_links










# get_group_B_indicators(driver)
def get_group_B_indicators(driver):
    url = "https://data.who.int/indicators"  
    driver.get(url)
    
    
    group_B_links = []
    indicators = driver.find_elements(By.CSS_SELECTOR, '.indicator-link-group-B')
    
    for indicator in indicators:
        link = indicator.get_attribute('href')
        group_B_links.append(link)
    
    return group_B_links









# get_indicator_data_for_group_A(driver)
# group A example https://data.who.int/indicators/i/49AC786/77D059C?m49=004

def get_indicator_data_for_group_A(driver, url):
    driver.get(url)
    
    # Assuming the data table has two columns (year, count)
    rows = driver.find_elements(By.CSS_SELECTOR, '.data-row')  # Update based on actual page structure
    data = []
    
    for row in rows:
        year = row.find_element(By.CSS_SELECTOR, '.year').text
        count = row.find_element(By.CSS_SELECTOR, '.count').text
        data.append({'year': year, 'count': count})
    
    return data

# get_indicator_data_for_group_ B(driver)
# group B example https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004
def get_indicator_data_for_group_B(driver, url):
    driver.get(url)
    
    # Example: Extract line graph data
    years = driver.find_elements(By.CSS_SELECTOR, '.graph-year')
    values = driver.find_elements(By.CSS_SELECTOR, '.graph-value')
    
    data = []
    
    for i in range(len(years)):
        year = years[i].text
        value = values[i].text
        data.append({'year': year, 'value': value})
    
    return data



In [35]:
# Import the necessary functions from functions.py
from functions import get_group_A_indicators, get_group_B_indicators, get_indicator_data_for_group_A, get_indicator_data_for_group_B

# Example usage
group_A_links = get_group_A_indicators(driver)
print(group_A_links)

group_B_links = get_group_B_indicators(driver)
print(group_B_links)

# Get data for a specific group A indicator
url_A = "https://data.who.int/indicators/i/49AC786/77D059C?m49=004"
group_A_data = get_indicator_data_for_group_A(driver, url_A)
print(group_A_data)

# Get data for a specific group B indicator
url_B = "https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004"
group_B_data = get_indicator_data_for_group_B(driver, url_B)
print(group_B_data)


None
None


TypeError: get_indicator_data_for_group_A() takes 1 positional argument but 2 were given

In [33]:
if __name__ == "__main__":
    
    # Get Group A indicators
    group_A_links = get_group_A_indicators(driver)
    
    # Extract data for a Group A indicator
    for link in group_A_links:
        data = get_indicator_data_for_group_A(driver, link)
        print(data)
    
    # Get Group B indicators
    group_B_links = get_group_B_indicators(driver)
    
    # Extract data for a Group B indicator
    for link in group_B_links:
        data = get_indicator_data_for_group_B(driver, link)
        print(data)
    
    driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=53653): Max retries exceeded with url: /session/8a6325d7c0ff15ebc0e0862a39544797/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002120DB72F00>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))